# **Virtual Tensile Tester" using Machine Learning**


# **1. Load the dataset**

In [ ]:
import pandas as pd
# 1. Load the dataset from a valid GitHub source
# This dataset contains 312 steel alloys with their chemical composition and strength
url = "https://raw.githubusercontent.com/batiukmaks/Steel-Strength-Prediction/master/steel_strength.csv"

df = pd.read_csv(url)


# **2. Exploratory Data Analysis (EDA)**





**Inspectthe data**

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# 2. Check for missing values
print("\n--- Missing Values ---")
print(df.isnull().sum())



In [ ]:
# Assuming 'df' is the DataFrame loaded in Step 1

print("Original dataset size:", df.shape)

# 1. Handle Missing Values in 'elongation'
# We drop the rows where 'elongation' is NaN.
df_clean = df.dropna(subset=['elongation'])

print("Dataset size after dropping 9 missing 'elongation' rows:", df_clean.shape)

In [ ]:
# Recheck for missing values
print("\n--- Missing Values ---")
print(df_clean.isnull().sum())

In [ ]:
df=df_clean

In [ ]:
df.info()



**Basic statistics**



In [ ]:
df.describe()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
sns.histplot(df['yield strength'], kde=True)
plt.title('Distribution of Steel Yield Strength')
plt.xlabel('Yield Strength (MPa)')
plt.show()

**Quiz!**

How about distribution of Tensile Strength and Elongation?

In [ ]:
df.hist(figsize=(17,12))

In [ ]:
plt.figure(figsize=(20,8))
sns.boxplot(data=df.drop(columns=['yield strength', 'tensile strength', 'elongation']), showfliers=True, orient="h")
plt.title("Boxplot of columns")

# **Correlation**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Create a numeric-only DataFrame by dropping the 'formula' column
#    Alternatively, you can use df.corr(numeric_only=True) in newer pandas versions
df_numeric = df.drop(columns=['formula'])

# 2. Calculate the correlation matrix
plt.figure(figsize=(12, 10))
corr_matrix = df_numeric.corr()

# 3. Plot the heatmap
#    annot=True displays the actual correlation values in the cells
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Feature Correlation Matrix')
plt.show()

**Correlation with yield strength**


In [ ]:
df_y = df.drop(columns=['formula'])
df_y.corrwith(df["yield strength"])

# **3. Data Splitting**

# **How to predict 'Yield Strength' (y) using the chemical elements (X)**

**1. Define the Target (y)**

**2. Define the Features (X)**

We will drop the target column to keep only the ingredients


In [ ]:
# 1. Define the Target (y)
y = df['yield strength']

# 2. Define the Features (X) - SELECTING ONLY METALS
# We create a list of only the metal columns
metal_features = ['mn','si','cr', 'ni', 'mo', 'v', 'n', 'nb', 'co', 'w', 'al', 'ti']

# Select only those columns from the dataframe
X = df[metal_features]

print("Selected only metal features:")
print(X.head())

D = df.drop(columns=['formula','yield strength'])
print("We have", D.shape[0], "steel samples.")
print("We are using", D.shape[1], "ingredients to predict strength.")

# **4. Train Model with RandomForestRegressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# 1. Split data into "Study Guide" (Train) and "Final Exam" (Test)
# We replace ____ with 0.2 (which means 20% of data is for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Initialize the AI Brain
model = RandomForestRegressor()

# 3. Train the model (Fit)
print("Training the ML model...")
# We replace ____ with X_train and y_train
model.fit(X_train, y_train)

print("Training Complete!")

# **6. Medel evaluation**

In [ ]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    median_absolute_error,
    max_error,
    explained_variance_score
)
import numpy as np

# 1. Get predictions
predictions = model.predict(X_test)

# 2. Calculate All Metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)
med_ae = median_absolute_error(y_test, predictions)
max_err = max_error(y_test, predictions)

# 3. Print the Report Card
print("=== MODEL PERFORMANCE REPORT ===")
print(f"1. R2 Score (Accuracy):          {r2:.4f}  ")
print(f"2. MAE (Average Error):          {mae:.2f} MPa ")
print(f"3. RMSE (Penalizes Big Errors):  {rmse:.2f} MPa ")
print(f"4. Median Error (Typical Error): {med_ae:.2f} MPa ")
print(f"5. Max Error (Worst Case):       {max_err:.2f} MPa ")
print("================================")

In [ ]:
import matplotlib.pyplot as plt

# 1. Get predictions for BOTH sets
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

plt.figure(figsize=(9, 9))

# 2. Plot Training Data (Blue)
# We use a lower 'alpha' (transparency) because there are usually many more training points
plt.scatter(y_train, train_preds, color='skyblue', alpha=0.4, label='Training Set (Memorized)')

# 3. Plot Test Data (Green)
# These are the "New" alloys the model has never seen
plt.scatter(y_test, test_preds, color='tomato', alpha=0.3, label='Test Set (New Data)')

# 4. Draw the "Perfect Prediction" Line
min_val = min(y.min(), train_preds.min()) # Find the lowest value in the whole dataset
max_val = max(y.max(), train_preds.max()) # Find the highest value
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=3, label='Perfect Match')

plt.title("Overfitting Check: Train vs. Test")
plt.xlabel("Actual Strength (MPa)")
plt.ylabel("Predicted Strength (MPa)")
plt.legend()
plt.grid(True)
plt.show()

# **7. Model comparison**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd

# 1. Setup the 6 Contenders
models = {
    "1. Linear Regression": LinearRegression(),
    "2. K-Nearest Neighbors": make_pipeline(StandardScaler(), KNeighborsRegressor()),
    "3. Random Forest": RandomForestRegressor(random_state=42),
    "4. Gradient Boosting (sklearn)": GradientBoostingRegressor(random_state=42),
    "5. Neural Network": make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(100,50), max_iter=5000, random_state=42)),
    "6. XGBoost": XGBRegressor(random_state=42)
}

# 2. Create a Scoreboard
results = []

print("Training 6 models... this might take a minute.\n")

# 3. The Training Loop
for name, model in models.items():
    # Train
    model.fit(X_train, y_train)

    # Predict
    preds = model.predict(X_test)

    # Score
    r2 = r2_score(y_test, preds)
    mae = mean_absolute_error(y_test, preds)

    # Save results
    results.append({"Model": name, "Accuracy (R2)": r2, "MAE (MPa)": mae})

# 4. Display the Final Leaderboard
leaderboard = pd.DataFrame(results).sort_values(by="Accuracy (R2)", ascending=False)
print(leaderboard.to_string(index=False, float_format='{:.4f}'.format))

In [ ]:
import joblib

# 1. Identify the Winner
# Grab the name of the model in the first row (since it's sorted by R2 descending)
best_model_name = leaderboard.iloc[0]['Model']
print(f"\n🏆 The Best Model is: {best_model_name}")

# 2. Retrieve the Trained Object
# Look up the actual model object from your original 'models' dictionary
best_model = models[best_model_name]

# 3. Save to Disk
filename = 'best_concrete_model_ST.pkl'
joblib.dump(best_model, filename)

print(f"✅ Successfully saved {best_model_name} to '{filename}'")

# **8. Feature importance**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 1. Load the Model
# Make sure this matches the filename you saved earlier
model = joblib.load('best_concrete_model_ST.pkl')
print(f"Loaded model type: {type(model)}")

# 1. Get the "Importance" numbers from the model
importances = model.feature_importances_

# 2. Create a DataFrame to display them nicely
feature_importance_df = pd.DataFrame({
    'Ingredient': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 3. Plot the Top 10 Ingredients
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Ingredient'][:10], feature_importance_df['Importance'][:10], color='teal')
plt.xlabel("Importance Score")
plt.title("Which Ingredients Drive Strength?")
plt.gca().invert_yaxis() # Put the most important at the top
plt.show()

# **9. DATA Prediction**

In [ ]:
import pandas as pd
import joblib

# 1. Load the Model (You likely already did this)
model = joblib.load('best_concrete_model_ST.pkl')

# 2. Define your features exactly as they were used in training
metal_features = ['mn', 'si', 'cr', 'ni', 'mo', 'v', 'n', 'nb', 'co', 'w', 'al', 'ti']

# 3. Create the single data point
# Replace these values with the actual numbers you want to predict
single_data_dict = {
    'mn': 0.5,
    'si': 4.5,
    'cr': 3.0,
    'ni': 16.0,
    'mo': 4.0,
    'v':  3.0,
    'n':  0.5,
    'nb': 2.0,
    'co': 3.0,
    'w':  1.0,
    'al': 2.0,
    'ti': 1.0
}

# 4. Convert to a DataFrame
# We wrap the dictionary in a list [ ... ] to make it a single row
X_single = pd.DataFrame([single_data_dict])

# Ensure columns are in the exact same order as training
X_single = X_single[metal_features]

# 5. Make the Prediction
prediction = model.predict(X_single)

# 6. Output the result
print(f"Input Features:\n{X_single}")
print("-" * 30)
print(f"Predicted Value (y): {prediction[0]:.4f}")

# **10. Parameter Opimization**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Define "Unexplored Territory"
# We allow the ingredients to go 50% higher than the maximum we've ever tested
# range_extension = 1.5 means "150% of the max value"
extended_max = X.max()
extended_min = X.min() # Keep min at 0 (can't have negative ingredients)

# 2. Generate 10,000 "Extreme" Alloys
num_samples = 10000
extreme_alloys = pd.DataFrame(
    np.random.uniform(extended_min, extended_max, size=(num_samples, len(X.columns))),
    columns=X.columns
)

# 3. Predict using a model capable of extrapolation
# Note: Random Forest generally CANNOT predict higher than training max.
# We will use Linear Regression here just to demonstrate "Unbounded" prediction,
# OR we rely on the Random Forest finding a better COMBINATION within the bounds.
# Let's stick to our best model (likely Gradient Boosting or RF) and see if it finds a peak.
predicted_extreme = model.predict(extreme_alloys)

# 4. Find the Winner
best_idx = np.argmax(predicted_extreme)
best_alloy = extreme_alloys.iloc[best_idx]
best_strength = predicted_extreme[best_idx]

print(f"=== THE EXTREME ALLOY ===")
print(f"Max Strength in Training Data: {y.max():.2f} MPa")
print(f"Predicted Extreme Strength:    {best_strength:.2f} MPa")

if best_strength > y.max():
    print("🚀 SUCCESS: We found a theoretical material stronger than anything existing!")
else:
    print("⚠️ LIMIT REACHED: The model thinks we've already hit the physical limit of steel.")

# 5. Visualize "The Edge of Knowledge"
plt.figure(figsize=(10, 6))
plt.hist(predicted_extreme, bins=30, alpha=0.5, label='AI Predictions (Unknown)', color='Orange')
plt.hist(y, bins=30, alpha=0.7, label='Real Data (Known)', color='blue')

plt.axvline(y.max(), color='red', linestyle='--', label='Current Record')
plt.legend()
plt.title("Did we break the World Record?")
plt.xlabel("Yield Strength (MPa)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# 6. Show the Top 5 Candidates
# Create a temporary table combining the Recipe with the Strength
results_table = extreme_alloys.copy()
results_table['Predicted Strength (MPa)'] = predicted_extreme

# Sort by Strength (Highest to Lowest) and take the top 5
top_5 = results_table.sort_values(by='Predicted Strength (MPa)', ascending=False).head(5)

print("\n=== TOP 5 CANDIDATE ALLOYS ===")
# We transpose (.T) the table so it's easier to read the chemical ingredients
print(top_5.T)